In [2]:
from ultralytics import YOLO
from PIL import Image
import cv2
import math

In [4]:
# create a VideoCapture object and set camera resolution to 640x480
cap = cv2.VideoCapture(0)
#cap.set(3, 640)
#cap.set(4, 480)

In [5]:
# load pretrained model
#model = YOLO("yolo-Weights/yolov8n.pt")
model = YOLO('yolov8n-pose.pt')

# train model
results = model.train(data='coco8-pose.yaml', epochs=100, imgsz=640)
metrics = model.val()  # evaluate model performance on the validation set

New https://pypi.org/project/ultralytics/8.0.205 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.201  Python-3.10.11 torch-2.1.0+cpu CPU (Intel Core(TM) i5-1035G1 1.00GHz)
engine\trainer: task=pose, mode=train, model=yolov8n-pose.pt, data=coco8-pose.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, 

In [ ]:
# Object detection 

# object classes
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]


while True:
    success, img = cap.read()
    results = model(img, stream=True)

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # confidence
            confidence = math.ceil((box.conf[0]*100))/100
            print("Confidence --->",confidence)

            # class name
            cls = int(box.cls[0])
            print("Class name -->", classNames[cls])

            # object details
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2

            cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [6]:
# Tracking
# https://docs.ultralytics.com/modes/track/#plotting-tracks-over-time

# Loop through the video frames
video_path = "videos\pexels_soccer_shoot.mp4"
cap = cv2.VideoCapture(0)

while True:
    # read from webcam
    success, img = cap.read()


    if success:
        # Run YOLOv8 inference on the frame
        #results = model(img, stream=True)
        results = model.track(img, persist=True)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Inference", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 480x640 1 person, 227.5ms
Speed: 2.0ms preprocess, 227.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 126.5ms
Speed: 2.0ms preprocess, 126.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 141.1ms
Speed: 2.0ms preprocess, 141.1ms inference, 14.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 128.8ms
Speed: 1.0ms preprocess, 128.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 128.1ms
Speed: 3.0ms preprocess, 128.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 128.5ms
Speed: 2.0ms preprocess, 128.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 135.8ms
Speed: 2.0ms preprocess, 135.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 133.1ms
Speed: 2.0ms preprocess, 133.1ms inference, 1.0ms postprocess per image a

In [4]:
# even simpler detection of body parts from webcam

model_2 = YOLO('yolov8m-pose.pt')
results = model_2(source=0, show = True, conf = 0.3, save = True)



1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 480x640 1 person, 514.0ms
0: 480x640 1 person, 509.4ms
0: 480x640 1 person, 519.0ms
0: 480x640 1 person, 650.5ms
0: 480x640 1 person, 550.9ms
0: 480x640 1 person, 487.9ms
0: 480x640 1 person, 514.2ms
0: 480x640 1 person, 497.8ms
0: 480x640 1 person, 522.7ms
0: 480x640 1 person, 494.3ms
0: 480x640 1 person, 510.9ms
0: 480x640 1 person, 515.3ms
0: 480x640 1 person, 500.5ms
0: 480x640 1 perso

KeyboardInterrupt: 

: 